# GTHA housing market database
# OSEMN methodology Step 2: Scrub
# Step 2.3 Spatial join of Teranet points with parcel-level land use from DMTI

---

This notebook describes Step 2.3 (part of _Step 2: Scrub_ of OSEMN methodology) performed on the Teranet dataset.

Step 2.3 involves the spatial join of Teranet points with parcel-level land use from DMTI.

Previous steps included: 

* Step 2.1 

    * the spatial join between the Teranet points and the polygons of GTHA Dissemination Areas (DAs)
    
    * During step 2.1, Teranet records whose coordinates fall outside of the GTHA boundary (as defined by the DA geometry) have been filtered out (6,803,691 of the original 9,039,241 Teranet records remain in the dataset)
     
    * In addition to that, three new columns (`OBJECTID`, `DAUID`, and `CSDNAME`) derived from DA attributes have been added to each Teranet transaction

---

For description of OSEMN methodology, see `methodology/0.osemn/osemn.pdf`.

For background information, description of the Teranet dataset, and its attributes, see `methodology/1.obtain/obtain.pdf`.

For description of _Step 2: Scrub_ of OSEMN methodology, see `methodology/2.scrub/scrub.pdf`.

For description of the cleanup plan for Teranet dataset, see `methodology/2.scrub/teranet_cleanup_plan.pdf`.

For description of Step 2.1 of the cleanup process, see `notebooks/2.scrub/2.1_teranet_gtha_spatial_join.ipynb`.


## Import dependencies

In [1]:
%matplotlib inline
import pandas as pd 
import geopandas as gpd
import os
from shapely.geometry import Point
from glob import glob
from time import time

In [2]:
data_path = '../../data/'
teranet_path = data_path + 'teranet/'
os.listdir(teranet_path)

['1.1_Teranet_DA.csv',
 '1.3_Teranet_DA_TAZ_PG_FSA.csv',
 'parcel16_epoi13.csv',
 '1.2_Teranet_DA_TAZ.csv',
 '1.4_Teranet_DA_TAZ_FSA_LU_LUDMTI.csv',
 '1.4_Teranet_DA_TAZ_FSA_LU.csv',
 '.ipynb_checkpoints',
 'ParcelLandUse.zip',
 'ParcelLandUse',
 'HHSaleHistory.csv',
 'GTAjoinedLanduseSales']

In [3]:
dmti_lu_path = data_path + 'dmti/dmti_lu_gtha/'
os.listdir(dmti_lu_path)

['lu_gtha02.shx',
 'lu_gtha10.shp',
 'lu_gtha10.shx',
 'lu_gtha03.sbx',
 'lu_gtha12.shp.xml',
 'lu_gtha02.prj',
 'lu_gtha13.shp.xml',
 'lu_gtha14.shp',
 'lu_gtha12.shx',
 'lu_gtha12.shp',
 'lu_gtha05.shp',
 'lu_gtha14.dbf',
 'lu_gtha02.dbf',
 'lu_gtha11.shp.xml',
 'lu_gtha09.cpg',
 'lu_gtha14.cpg',
 'lu_gtha01.prj',
 'lu_gtha11.cpg',
 'lu_gtha07.dbf',
 'lu_gtha14.sbn',
 'lu_gtha14.shx',
 'lu_gtha12.dbf',
 'lu_gtha09.dbf',
 'lu_gtha12.cpg',
 'lu_gtha05.sbx',
 'lu_gtha06.prj',
 'lu_gtha11.sbn',
 'lu_gtha10.sbx',
 'lu_gtha01.sbn',
 'lu_gtha08.cpg',
 'lu_gtha09.prj',
 'lu_gtha09.shp.xml',
 'lu_gtha07.sbx',
 'lu_gtha13.shp',
 'lu_gtha10.shp.xml',
 'lu_gtha12.prj',
 'lu_gtha05.sbn',
 'lu_gtha09.sbn',
 'lu_gtha12.sbn',
 'lu_gtha04.sbn',
 'lu_gtha07.prj',
 'lu_gtha08.sbx',
 'lu_gtha11.shp',
 'lu_gtha03.prj',
 'lu_gtha13.dbf',
 'lu_gtha03.shx',
 'lu_gtha07.shp.xml',
 'lu_gtha06.shp.xml',
 'lu_gtha10.cpg',
 'lu_gtha05.cpg',
 'lu_gtha07.shx',
 'lu_gtha01.shp.xml',
 'lu_gtha04.shp.xml',
 'lu_gtha0

In [4]:
dmti_lu_shapefile_list = glob(dmti_lu_path + '*.shp')
dmti_lu_shapefile_list.sort()
dmti_lu_shapefile_list

['../../data/dmti/dmti_lu_gtha/lu_gtha01.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha02.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha03.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha04.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha05.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha06.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha07.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha08.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha09.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha10.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha11.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha12.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha13.shp',
 '../../data/dmti/dmti_lu_gtha/lu_gtha14.shp']

## Load Teranet data

In [5]:
t = time()
teranet_df = pd.read_csv(teranet_path + '1.4_Teranet_DA_TAZ_FSA_LU.csv',
                         parse_dates=['registration_date'])
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(teranet_df.shape[0], teranet_df.shape[1]) + 
      "\n-- Column names:\n", teranet_df.columns)

t = time()
# combine values in columns 'x' and 'y' into a POINT geometry object
geometry = [Point(xy) for xy in zip(teranet_df['X'], teranet_df['Y'])]
# generate a new GeoDataFrame by adding point geometry to data frame 'teranet_sales_data'
teranet_gdf = gpd.GeoDataFrame(teranet_df, geometry=geometry)
elapsed = time() - t
print("\n----- Geometry generated from 'X' and 'Y' pairs, GeoDataFrame created!"
      "\nin {0:.2f} seconds ({1:.2f} minutes)".format(elapsed, elapsed / 60) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(teranet_gdf.shape[0], teranet_gdf.shape[1]) + 
      "\n-- Column names:\n", teranet_gdf.columns)

# add CRS for WGS84 (lat-long) to GeoDataFrame with Teranet records
teranet_gdf.crs = {'proj': 'latlong', 
                   'ellps': 'WGS84', 
                   'datum': 'WGS84', 
                   'no_defs': True}
print("\n----- CRS dictionary for WGS84 added to geo data frame 'teranet_gdf'!")

----- DataFrame loaded
in 34.96 seconds
with 6,803,691 rows
and 26 columns
-- Column names:
 Index(['transaction_id', 'lro_num', 'pin', 'consideration_amt',
       'registration_date', 'POSTAL_CODE', 'PROVINCE', 'UNITNO', 'STREET_NAME',
       'STREET_DESIGNATION', 'STREET_DIRECTION', 'MUNICIPALITY',
       'STREET_SUFFIX', 'STREET_NUMBER', 'X', 'Y', 'DAUID', 'CSDUID',
       'CSDNAME', 'TAZ_O', 'FSA', 'PCA_ID', 'postal_code_dmti', 'pin_lu',
       'LANDUSE', 'PROP_CODE'],
      dtype='object')

----- Geometry generated from 'X' and 'Y' pairs, GeoDataFrame created!
in 90.33 seconds (1.51 minutes)
with 6,803,691 rows
and 27 columns
-- Column names:
 Index(['transaction_id', 'lro_num', 'pin', 'consideration_amt',
       'registration_date', 'POSTAL_CODE', 'PROVINCE', 'UNITNO', 'STREET_NAME',
       'STREET_DESIGNATION', 'STREET_DIRECTION', 'MUNICIPALITY',
       'STREET_SUFFIX', 'STREET_NUMBER', 'X', 'Y', 'DAUID', 'CSDUID',
       'CSDNAME', 'TAZ_O', 'FSA', 'PCA_ID', 'postal_code_dmti', 

## Perform the spatial join of Teranet points with DMTI land use parcels by year
### Validate projections
>Note: EPSG:4326 and WGS 84 represent the [same projection](https://spatialreference.org/ref/epsg/wgs-84/).

In [6]:
teranet_lu_gdf = gpd.GeoDataFrame()

for shapefile in dmti_lu_shapefile_list:
    # read one year of DMTI land use data
    lu_gdf = gpd.read_file(shapefile)
    cols = ['CATEGORY', 'geometry']
    lu_gdf = lu_gdf[cols]
    lu_gdf = lu_gdf.to_crs(teranet_gdf.crs)
    
    # take a subset of Teranet records for that year
    year = int('20' + shapefile[-6:-4])
    mask1 = teranet_gdf['registration_date'].dt.year == year
    s = teranet_gdf[mask1]
    
    # perform the spatial join of Teranet points with DMTI land use polygons
    print("\n----- {0}\nJoining {1:,} land use polygons from DMTI with {2:,} Teranet points..."
          .format(year, len(lu_gdf), len(s)) + 
         "\nCRS of DMTI land use :", lu_gdf.crs, "\nCRS of Teranet points:", s.crs)
    t = time()
    teranet_lu_sjoin_gdf = gpd.sjoin(s, lu_gdf, 
                                     how='left', op='within')
    teranet_lu_gdf = teranet_lu_gdf.append(teranet_lu_sjoin_gdf)
    elapsed = time() - t
    print("\nSpatial join completed in {0:,.2f} seconds ({1:,.2f} minutes)\n{2:,} rows in the resultant GeoDataFrame"
          .format(elapsed, elapsed / 60, len(teranet_lu_gdf)))


----- 2001
Joining 56,363 land use polygons from DMTI with 215,620 Teranet points...
CRS of DMTI land use : {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True} 
CRS of Teranet points: {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}

Spatial join completed in 26.94 seconds (0.45 minutes)
215,620 rows in the resultant GeoDataFrame

----- 2002
Joining 56,363 land use polygons from DMTI with 241,671 Teranet points...
CRS of DMTI land use : {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True} 
CRS of Teranet points: {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}

Spatial join completed in 30.16 seconds (0.50 minutes)
457,291 rows in the resultant GeoDataFrame

----- 2003
Joining 73,529 land use polygons from DMTI with 256,055 Teranet points...
CRS of DMTI land use : {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True} 
CRS of Teranet points: {'proj': 'latlong', 'ellps': 'WGS84', 

In [7]:
teranet_lu_gdf['CATEGORY'].value_counts()

Residential                     2205585
Open Area                        584460
Resource and Industrial          485892
Commercial                       107305
Government and Institutional      96255
Parks and Recreational            53068
Waterbody                          3492
resource and Industrial              20
Name: CATEGORY, dtype: int64

In [8]:
teranet_lu_gdf['registration_date'].dt.year.value_counts().sort_index()

2001    215620
2002    241671
2003    256055
2004    263662
2005    259811
2006    261836
2007    276892
2008    240614
2009    233051
2010    256125
2011    261220
2012    249857
2013    249309
2014    270431
Name: registration_date, dtype: int64

### Join DMTI land use columns back to the full Teranet dataset
Teranet records from years for which there is no DMTI land use data available will have `dmti_lu` value of NaN.

In [9]:
teranet_lu_gdf = teranet_lu_gdf[['transaction_id', 'CATEGORY']].rename(columns={'CATEGORY': 'dmti_lu'})
teranet_full_lu_df = pd.merge(teranet_gdf, teranet_lu_gdf, how='left', 
                               left_on='transaction_id', right_on='transaction_id')
print("New columns were merged to the full Teranet GeoDataFrame.")

New columns were merged to the full Teranet GeoDataFrame.


In [10]:
teranet_full_lu_df['dmti_lu'].value_counts()

Residential                     2205585
Open Area                        584460
Resource and Industrial          485892
Commercial                       107305
Government and Institutional      96255
Parks and Recreational            53068
Waterbody                          3492
resource and Industrial              20
Name: dmti_lu, dtype: int64

In [11]:
mask1 = teranet_full_lu_df['dmti_lu'].isnull()
count_full = teranet_full_lu_df.loc[~mask1, 'registration_date'].dt.year.value_counts().sort_index()
count_full

2001    215620
2002    241669
2003    256053
2004    263657
2005    259809
2006    261834
2007    276891
2008    240613
2009    233051
2010    256122
2011    261219
2012    249856
2013    249307
2014    270376
Name: registration_date, dtype: int64

### Validate results

In [12]:
len(teranet_lu_gdf)

3536154

In [13]:
teranet_lu_gdf['transaction_id'].nunique()

3536154

In [14]:
mask1 = teranet_lu_gdf['transaction_id'].isin(teranet_full_lu_df['transaction_id'])
len(teranet_lu_gdf[mask1])

3536154

In [15]:
mask1 = teranet_full_lu_df['dmti_lu'].isnull()
len(teranet_full_lu_df[~mask1])

3536077

Slightly smaller number of Teranet records in the full merged dataset show non-null `dmti_lu` because 77 values of `CATEGORY` were null in the original DMTI shapefiles.

In [16]:
teranet_lu_gdf['dmti_lu'].isnull().sum()

77

### Save results to a .csv file

In [17]:
teranet_full_lu_df.shape

(6803691, 28)

In [18]:
teranet_full_lu_df.columns

Index(['transaction_id', 'lro_num', 'pin', 'consideration_amt',
       'registration_date', 'POSTAL_CODE', 'PROVINCE', 'UNITNO', 'STREET_NAME',
       'STREET_DESIGNATION', 'STREET_DIRECTION', 'MUNICIPALITY',
       'STREET_SUFFIX', 'STREET_NUMBER', 'X', 'Y', 'DAUID', 'CSDUID',
       'CSDNAME', 'TAZ_O', 'FSA', 'PCA_ID', 'postal_code_dmti', 'pin_lu',
       'LANDUSE', 'PROP_CODE', 'geometry', 'dmti_lu'],
      dtype='object')

In [19]:
save_path = teranet_path + '1.4_Teranet_DA_TAZ_FSA_LU_LUDMTI.csv'
t = time()
teranet_full_lu_df.drop('geometry', axis=1).to_csv(save_path, index=False)
elapsed = time() - t
print("DataFrame saved to file:\n", save_path,
      "\ntook {0:.2f} seconds ({1:.2f} minutes)".format(elapsed, elapsed / 60))

DataFrame saved to file:
 ../../data/teranet/1.4_Teranet_DA_TAZ_FSA_LU_LUDMTI.csv 
took 248.09 seconds (4.13 minutes)
